# Load Data

In [1]:
from preprocess import preprocess_pipeline

data = preprocess_pipeline()

2025-08-23 18:22:18.207797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755962538.221164  180302 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755962538.225296  180302 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755962538.236841  180302 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755962538.236859  180302 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755962538.236861  180302 computation_placer.cc:177] computation placer alr

1. Include only first admissions: N=32513
2. Exclusion by ages: N=25548
3. Include only patients who admitted for at least 24 hours: N=25168
4. Exclude patients who died within 54-hours of admission: N=24556


/home/bnet/ronsheinin/MLHC/MLHC/preprocess.py:210: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pivot = pd.pivot_table(merged, index=['subject_id', 'hadm_id', pd.Grouper(key='charttime', freq=PREDICT_FREQ)],
/home/bnet/ronsheinin/MLHC/MLHC/preprocess.py:270: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], axis=1, inplace=True)
/home/bnet/ronsheinin/MLHC/MLHC/preprocess.py:274: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], a

Pooled embeddings shape: 24529
Number of subjects: 24529


# Datasets creation

In [3]:
data["prescriptions_train"].shape[0] == data["padded_tensor_train"].shape[0]

True

In [11]:
import Dataset
import imp
imp.reload(Dataset)
import torch
from torch.utils.data import DataLoader
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k=10

train_labels = torch.tensor(data["y_train"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
val_labels = torch.tensor(data["y_val"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
test_labels = torch.tensor(data["y_test"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)


batch_size = 256
datasets = {x: Dataset.PatientDataset(d, y, core=data["padded_tensor_core"], padding_mask=m, padding_mask_core=data["padding_mask_core"], k=k ,notes=n, bios=b, pres=p) for x, d, y, m, n, b, p in
        zip(['train', 'val', 'test'], [data["padded_tensor_train"], data["padded_tensor_val"], data["padded_tensor_test"]],
            [train_labels, val_labels, test_labels],
            [data["padding_mask_train"], data["padding_mask_val"], data["padding_mask_test"]],
            [torch.stack(data["notes_df_train"].embeddings.values.tolist()),
             torch.stack(data["notes_df_val"].embeddings.values.tolist()),
             torch.stack(data["notes_df_test"].embeddings.values.tolist())],
             [torch.tensor(data["bio_train"].values >= 1, dtype=torch.float32).to(DEVICE),
              torch.tensor(data["bio_val"].values >= 1, dtype=torch.float32).to(DEVICE),
              torch.tensor(data["bio_test"].values >= 1, dtype=torch.float32).to(DEVICE)],
              [torch.tensor(data["prescriptions_train"].values, dtype=torch.float32).to(DEVICE),
               torch.tensor(data["prescriptions_val"].values, dtype=torch.float32).to(DEVICE),
               torch.tensor(data["prescriptions_test"].values, dtype=torch.float32).to(DEVICE)])}
dataloaders = {x: DataLoader(datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val', 'test']}

# Model Training

In [23]:
import Model 
import imp
imp.reload(Model)
model = Model.GraphGRUMortalityModel(input_dim=data["padded_tensor_train"].shape[2], hidden_dim=64, n1_gat_layers=1, n2_gru_layers=2, X_core=data["padded_tensor_core"], num_of_bios=data["bio_train"].shape[1],num_prescriptions=data["prescriptions_train"].shape[1], 
                                     bios_hidden_dim=16, pres_hidden_dim=32, core_padding_mask=data["padding_mask_core"], num_heads=4, dropout=0.1, seq_len=data["padded_tensor_train"].shape[1], k=k, gnn_flag=True).to(DEVICE)

model.train_all(dataloaders, datasets, epochs=10, learning_rate=1e-3)
print("Training completed. Validating on test set...")

model.validate(dataloaders['test'], datasets['test'])
#print(f'Test Accuracy in model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')

Pos weight 0: 7.5538
Pos weight 1: 1.1253
Pos weight 2: 23.8500
Starting epoch 1/10


  0%|          | 0/76 [00:00<?, ?it/s]

100%|██████████| 76/76 [00:59<00:00,  1.28it/s]


Epoch 1/10, Loss: 3.0955


100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


...

Epoch 10/10, Loss: 2.4269


100%|██████████| 10/10 [00:08<00:00,  1.23it/s]


Val Mortality - AUC: 0.8257 | AP: 0.4205
Val Prolonged LOS - AUC: 0.7933 | AP: 0.7728
Val Readmission - AUC: 0.6176 | AP: 0.0660
Best model updated
Training completed. Validating on test set...


100%|██████████| 10/10 [00:06<00:00,  1.43it/s]


(0.8524343593257856,
 0.522004393155386,
 0.8175285502397754,
 0.798777499280743,
 0.6243221102091387,
 0.08293448825860515)

In [24]:
res = model.validate(dataloaders['test'], datasets['test'])

print(f"Test Mortality: AUC-ROC: {res[0]:.2f}  AP: {res[1]:.2f}")
print(f"Test Prolonged Stay: AUC-ROC: {res[2]:.2f}  AP: {res[3]:.2f}")
print(f"Test Readmission: AUC-ROC: {res[4]:.2f}  AP: {res[5 ]:.2f}")

100%|██████████| 10/10 [00:07<00:00,  1.38it/s]

Test Mortality: AUC-ROC: 0.85  AP: 0.52
Test Prolonged Stay: AUC-ROC: 0.82  AP: 0.80
Test Readmission: AUC-ROC: 0.62  AP: 0.08


In [9]:
model.validate(dataloaders['test'], datasets['test'])


100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


(0.862270225393303,
 0.5374146532039636,
 0.8114620700121142,
 0.7750227972338779,
 0.5760484176849882,
 0.05048034000235363)